In [229]:
import pandas
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from datetime import datetime
from pytz import timezone



In [3]:
!pip install tqdm
import tqdm
import numpy as np
max = 100000000000000000
for i in tqdm.tqdm(range(max)):
  np.pi*np.pi


  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)


100%|██████████| 10000/10000 [00:00<00:00, 1124568.73it/s]


In [230]:
model_lr = LogisticRegression(solver='liblinear')

In [231]:

from sklearn.model_selection import train_test_split
TEST_SIZE = 0.2
objectiv_col = "isFake"
#トレーニングデータと検証用のテストデータに分割
def split_data_frame(df:pandas.DataFrame, objectiv_col:str):
  df_train, df_val =train_test_split(df, test_size=TEST_SIZE, random_state=42)
  train_y = df_train[objectiv_col]
  train_x = df_train.drop(objectiv_col, axis=1)

  val_y = df_val[objectiv_col]
  val_x = df_val.drop(objectiv_col, axis=1)
  return [train_x, train_y, val_x, val_y]

train_x, train_y, val_x, val_y = split_data_frame(df_train, objectiv_col)

In [232]:
def clean_text(text):
  return text.replace(' ', '').replace('　', '').replace('__BR__', '\n').replace('\xa0', '').replace('\r', '').lstrip('\n')


df_hate_train = pandas.read_csv("hate/train.csv", index_col=0)
df_hate_train.drop("source", axis=1)
df_hate_test = pandas.read_csv("hate/test.csv", index_col=0)
df_hate_test.drop("source", axis=1)
df_hate_train['text'] = df_hate_train['text'].apply(clean_text)
df_hate_test['text'] = df_hate_test['text'].apply(clean_text)

all_texts = list(df_hate_train["text"])
all_texts.extend(list(df_hate_test["text"]))
janome_tokenizer = Tokenizer()
vectorizer = CountVectorizer(tokenizer=lambda text: janome_tokenizer.tokenize(text, wakati=True))


In [233]:
%%time
vectorizer.fit(all_texts)

c:\workspace\NeuroDive\.venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: total: 16.7 s
Wall time: 23.5 s


CountVectorizer(tokenizer=<function <lambda> at 0x00000232B09D35B0>)

In [234]:
model_lr = LogisticRegression(solver='liblinear')

In [235]:

train_x, train_y, val_x, val_y = split_data_frame(df_hate_train, "label")

In [236]:
%%time
train_text_vectors = vectorizer.transform(train_x['text'])
model_lr.fit(train_text_vectors, train_y)

CPU times: total: 8.95 s
Wall time: 11.6 s


LogisticRegression(solver='liblinear')

In [237]:
from sklearn.metrics import precision_score, recall_score, f1_score
def print_score(val_y, predict: pandas.DataFrame):
  print("正解率↓")
  print(accuracy_score(val_y, predict))
  print("適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓")
  print(precision_score(val_y, predict))
  print("再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓")
  print(recall_score(val_y, predict))
  print("F値↓")
  print(f1_score(val_y, predict))

In [238]:
%%time
test_text_vectors = vectorizer.transform(val_x['text'])
predict = model_lr.predict(test_text_vectors)
print_score(val_y, predict)

正解率↓
0.9505703422053232
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.5833333333333334
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.25
F値↓
0.35
CPU times: total: 2 s
Wall time: 2.77 s


In [239]:
%%time
df_tweet = pandas.read_csv("hate/tweet.csv", index_col=0)
df_tweet['text'] = df_tweet['text'].apply(clean_text)
test_tweet_vectors = vectorizer.transform(df_tweet['text'])

predict = model_lr.predict(test_tweet_vectors)
print_score(df_tweet["label"], predict)

正解率↓
0.7254901960784313
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.8888888888888888
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.38095238095238093
F値↓
0.5333333333333333
CPU times: total: 46.9 ms
Wall time: 121 ms


In [240]:
%%time
from sklearn.svm import LinearSVC
linear_svc_model = LinearSVC()
train_text_vectors = vectorizer.transform(train_x['text'])
linear_svc_model.fit(train_text_vectors, train_y)

CPU times: total: 8.2 s
Wall time: 12 s


c:\workspace\NeuroDive\.venv\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC()

In [241]:
%%time
predict = linear_svc_model.predict(test_text_vectors)
print_score(val_y, predict)

正解率↓
0.9458174904942965
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.48717948717948717
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.3392857142857143
F値↓
0.4
CPU times: total: 0 ns
Wall time: 8.93 ms


In [242]:
%%time
predict = linear_svc_model.predict(test_tweet_vectors)
print_score(df_tweet["label"], predict)

正解率↓
0.7843137254901961
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.9166666666666666
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.5238095238095238
F値↓
0.6666666666666666
CPU times: total: 0 ns
Wall time: 11.2 ms


In [243]:
%%time
from sklearn.neural_network import MLPClassifier
nn_model = MLPClassifier(hidden_layer_sizes=(16,))
nn_model.fit(train_text_vectors, train_y)

CPU times: total: 2.95 s
Wall time: 21.7 s


MLPClassifier(hidden_layer_sizes=(16,))

In [244]:
%%time
predict = nn_model.predict(test_text_vectors)
print_score(val_y, predict)

正解率↓
0.9372623574144486
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.34375
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.19642857142857142
F値↓
0.25
CPU times: total: 0 ns
Wall time: 13.3 ms


In [245]:
%%time
predict = nn_model.predict(test_tweet_vectors)
print_score(df_tweet["label"], predict)

正解率↓
0.803921568627451
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.9230769230769231
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.5714285714285714
F値↓
0.7058823529411765
CPU times: total: 0 ns
Wall time: 16.2 ms


In [248]:
%%time
from sklearn.neighbors import KNeighborsClassifier
kn_model = KNeighborsClassifier()
kn_model.fit(train_text_vectors, train_y)
print("-----------------------train.csvデータ----------------")
test_predict = kn_model.predict(test_text_vectors)
print_score(val_y, test_predict)
print("-----------------------tweet.csvデータ----------------")
tweet_predict = kn_model.predict(test_tweet_vectors)
print_score(df_tweet["label"], tweet_predict)

-----------------------train.csvデータ----------------
正解率↓
0.9467680608365019
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.0
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.0
F値↓
0.0
-----------------------tweet.csvデータ----------------
正解率↓
0.5686274509803921
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.0
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.0
F値↓
0.0
CPU times: total: 234 ms
Wall time: 384 ms


c:\workspace\NeuroDive\.venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [250]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(train_text_vectors, train_y)
print("-----------------------train.csvデータ----------------")
print_score(val_y, rf_model.predict(test_text_vectors))
print("-----------------------tweet.csvデータ----------------")
print_score(df_tweet["label"], rf_model.predict(test_tweet_vectors))

-----------------------train.csvデータ----------------
正解率↓
0.9448669201520913
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.0
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.0
F値↓
0.0
-----------------------tweet.csvデータ----------------
正解率↓
0.6078431372549019
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
1.0
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.047619047619047616
F値↓
0.09090909090909091
CPU times: total: 4.95 s
Wall time: 7.24 s


In [251]:
%%time
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(train_text_vectors, train_y)
print("-----------------------train.csvデータ----------------")
print_score(val_y, nb_model.predict(test_text_vectors))
print("-----------------------tweet.csvデータ----------------")
print_score(df_tweet["label"], nb_model.predict(test_tweet_vectors))

-----------------------train.csvデータ----------------
正解率↓
0.9391634980988594
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.0
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.0
F値↓
0.0
-----------------------tweet.csvデータ----------------
正解率↓
0.6274509803921569
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
1.0
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.09523809523809523
F値↓
0.17391304347826086
CPU times: total: 0 ns
Wall time: 26.2 ms


In [252]:
%%time
train_text_vectors = vectorizer.transform(train_x['text'])
model_lr.fit(train_text_vectors, train_y)
test_text_vectors = vectorizer.transform(val_x['text'])
print("-----------------------train.csvデータ----------------")
print_score(val_y, model_lr.predict(test_text_vectors))
print("-----------------------tweet.csvデータ----------------")
print_score(df_tweet["label"], model_lr.predict(test_tweet_vectors))

-----------------------train.csvデータ----------------
正解率↓
0.9505703422053232
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.5833333333333334
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.25
F値↓
0.35
-----------------------tweet.csvデータ----------------
正解率↓
0.7254901960784313
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.8888888888888888
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.38095238095238093
F値↓
0.5333333333333333
CPU times: total: 12.3 s
Wall time: 15.5 s


In [253]:
%%time
from sklearn.svm import LinearSVC
linear_svc_model = LinearSVC()
linear_svc_model.fit(train_text_vectors, train_y)
print("-----------------------train.csvデータ----------------")
print_score(val_y, linear_svc_model.predict(test_text_vectors))
print("-----------------------tweet.csvデータ----------------")
print_score(df_tweet["label"], linear_svc_model.predict(test_tweet_vectors))

-----------------------train.csvデータ----------------
正解率↓
0.9458174904942965
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.48717948717948717
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.3392857142857143
F値↓
0.4
-----------------------tweet.csvデータ----------------
正解率↓
0.7843137254901961
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.9166666666666666
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.5238095238095238
F値↓
0.6666666666666666
CPU times: total: 15.6 ms
Wall time: 42.5 ms


c:\workspace\NeuroDive\.venv\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [254]:
%%time
from sklearn.neural_network import MLPClassifier
nn_model = MLPClassifier(hidden_layer_sizes=(16,))
nn_model.fit(train_text_vectors, train_y)
print("-----------------------train.csvデータ----------------")
print_score(val_y, nn_model.predict(test_text_vectors))
print("-----------------------tweet.csvデータ----------------")
print_score(df_tweet["label"], nn_model.predict(test_tweet_vectors))

-----------------------train.csvデータ----------------
正解率↓
0.9391634980988594
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.375
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.21428571428571427
F値↓
0.2727272727272727
-----------------------tweet.csvデータ----------------
正解率↓
0.803921568627451
適合率（これ低いとヘイトじゃない文章をヘイトと判定した率高い まあ低くても許容）↓
0.9230769230769231
再現率（これ低いとヘイト文章をヘイトじゃないと判定した率高い こっちは高くないといけない）↓
0.5714285714285714
F値↓
0.7058823529411765
CPU times: total: 2.22 s
Wall time: 16.6 s


# trainデータに、ヘイト文章のデータが極端に少ない偏りがある

# モデル→サポートベクトルマシンによる線形か、ニューラルネットワークがいい感じ
- TODO:LightGBMは？ xgboostは？

# TODO:特徴量エンジニアリングを行う
- TF-IDF
- Word2Vec
- FastText
- BERT
- LUKE